# Simple HigherHRNet
This is a light Google Colab notebook showing how to use the [simple-HigherHRNet](https://github.com/stefanopini/simple-HigherHRNet) repository.

It includes the conversion to TensorRT and a test of the converted model.
Please skip the section "TensorRT" if not interested.

## Pytorch

### Clone the repo and install the dependencies

In [22]:
# clone the repo
!git clone https://github.com/stefanopini/simple-HigherHRNet.git

Cloning into 'simple-HigherHRNet'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 120 (delta 47), reused 88 (delta 28), pack-reused 0
Receiving objects: 100% (120/120), 15.57 MiB | 8.21 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [23]:
%cd simple-HigherHRNet
!pwd

/content/simple-HigherHRNet
/content/simple-HigherHRNet


In [24]:
# checkout the tensorrt branch
!git checkout tensorrt

Branch 'tensorrt' set up to track remote branch 'tensorrt' from 'origin'.
Switched to a new branch 'tensorrt'


In [25]:
# install requirements
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
# install vlc to get video codecs
!apt install vlc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-freefont-ttf liba52-0.7.4 libaa1 libaribb24-0
  libbasicusageenvironment1 libcddb2 libdca0 libdvbpsi10 libdvdnav4
  libdvdread4 libebml4v5 libfaad2 libgroupsock8 libkate1 libldb1
  liblirc-client0 liblivemedia62 liblua5.2-0 libmad0 libmatroska6v5
  libmicrodns0 libmpcdec6 libmpeg2-4 libmtp-common libmtp-runtime libmtp9
  libnfs11 libnotify4 libopenmpt-modplug1 libplacebo4 libprotobuf-lite10
  libproxy-tools libqt5svg5 libqt5x11extras5 libresid-builder0c2a
  libsdl-image1.2 libsdl1.2debian libsecret-1-0 libsecret-common libshout3
  libsidplay2 libsmbclient libspeexdsp1 libtag1v5 libtag1v5-vanilla libtalloc2
  libtdb1 libtevent0 libupnp6 libusageenvironment3 libva-wayland2 libvlc-bin
  libvlc5 libvlcc

### Download weights and video

In [26]:
# download weights (pose_higher_hrnet_w32_512.pth)
!gdown 1V9Iz0ZYy9m8VeaspfKECDW0NKlGsYmO1
!mkdir weights
!mv /content/simple-HigherHRNet/pose_higher_hrnet_w32_512.pth /content/simple-HigherHRNet/weights/pose_higher_hrnet_w32_512.pth

Downloading...
From: https://drive.google.com/uc?id=1V9Iz0ZYy9m8VeaspfKECDW0NKlGsYmO1
To: /content/simple-HigherHRNet/pose_higher_hrnet_w32_512.pth
100% 115M/115M [00:00<00:00, 220MB/s]


In [27]:
# download a publicly available video (or just get your own)
!wget https://commondatastorage.googleapis.com/gtv-videos-bucket/sample/WeAreGoingOnBullrun.mp4

--2022-11-21 23:37:17--  https://commondatastorage.googleapis.com/gtv-videos-bucket/sample/WeAreGoingOnBullrun.mp4
Resolving commondatastorage.googleapis.com (commondatastorage.googleapis.com)... 142.251.10.128, 2404:6800:4003:c0f::80
Connecting to commondatastorage.googleapis.com (commondatastorage.googleapis.com)|142.251.10.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13183260 (13M) [video/mp4]
Saving to: ‘WeAreGoingOnBullrun.mp4’

WeAreGoingOnBullrun 100%[===================>]  12.57M  --.-KB/s    in 0.07s   

2022-11-21 23:37:18 (190 MB/s) - ‘WeAreGoingOnBullrun.mp4’ saved [13183260/13183260]



### Test the live script
This step can be skipped if interested in the TensorRT conversion.

In [29]:
!python ./scripts/live-demo.py --filename WeAreGoingOnBullrun.mp4 --save_video

23:43:27 ::    Helper     ::   INFO   :: Running VidGear Version: 0.2.6
device: 'cuda' - 1 GPU(s) will be used
/content/simple-HigherHRNet/misc/HeatmapParser.py:168: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  y = (ind // w).long()
framerate: 8.096502 fps / detected people: 0 /content/simple-HigherHRNet/misc/HeatmapParser.py:236: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, us

Now check out the video output.avi


## TensorRT
This section install TensorRT 8.5, converts the model to TensorRT (.engine) and tests the converted model.

Tested with TensorRT 8.5.1-1+cuda11.8 and python package tensorrt 8.5.1.7 .

### Install TensorRT
A GPU is needed for this step. Please change the runtime type to "GPU".


In [30]:
# check a GPU runtime is selected
!nvidia-smi

Mon Nov 21 23:47:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
%%bash
wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb

dpkg -i nvidia-machine-learning-repo-*.deb
apt-get update

sudo apt-get install libnvinfer8 python3-libnvinfer

(Reading database ... 123994 files and directories currently installed.)
Preparing to unpack nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb ...
Unpacking nvidia-machine-learning-repo-ubuntu1804 (1.0.0-1) over (1.0.0-1) ...
Setting up nvidia-machine-learning-repo-ubuntu1804 (1.0.0-1) ...
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelea

--2022-11-21 23:10:05--  https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.39.144
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.39.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2926 (2.9K) [application/x-deb]
Saving to: ‘nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb.1’

     0K ..                                                    100%  180M=0s

2022-11-21 23:10:05 (180 MB/s) - ‘nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb.1’ saved [2926/2926]

W: Target Packages (Packages) is configured multiple times in /etc/apt/sources.list:51 and /etc/apt/sources.list.d/nvidia-machine-learning.list:1
W: Target Packages (Packages) is configured multiple times in /etc/apt/sources.list:51 and /etc/apt/sources.list.d/nvidia-machine-le

In [6]:
# check TensorRT version
print("TensorRT version: ")
!dpkg -l | grep nvinfer

TensorRT version: 
ii  libnvinfer-plugin8                      8.5.1-1+cuda11.8                    amd64        TensorRT plugin libraries
ii  libnvinfer8                             8.5.1-1+cuda11.8                    amd64        TensorRT runtime libraries
ii  python3-libnvinfer                      8.5.1-1+cuda11.8                    amd64        Python 3 bindings for TensorRT


In [39]:
!pip install tensorrt onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.1 MB 24.5 MB/s 


### Export the model with tensorrt

In [40]:
!python scripts/export-tensorrt-model.py --device 0 --half

export-tensorrt-model: weights=./weights/pose_higher_hrnet_w32_512.pth, hrnet_c=32, hrnet_j=17, imgsz=[512, 960], batch_size=1, device=0, half=True, int8=False, dynamic=False, simplify=False, opset=12, verbose=False, workspace=1, include=['engine']
YOLOv5 🚀 2022-11-21 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)


ONNX: starting export with onnx 1.12.0...
ONNX: export success ✅ 52.5s, saved as weights/pose_higher_hrnet_w32_512.onnx (109.3 MB)

TensorRT: starting export with TensorRT 8.5.1.7...
[11/21/2022-23:54:02] [TRT] [I] [MemUsageChange] Init CUDA: CPU +0, GPU +0, now: CPU 0, GPU 1572 (MiB)
[11/21/2022-23:54:05] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +0, GPU +74, now: CPU 0, GPU 1646 (MiB)
[11/21/2022-23:54:05] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars
scripts/export-tensorrt-model.py:

### Test the tensorrt model

In [42]:
!python scripts/live-demo.py --enable_tensorrt --filename=WeAreGoingOnBullrun.mp4 --hrnet_weights='weights/pose_higher_hrnet_w32_512.engine' --save_video

00:02:18 ::    Helper     ::   INFO   :: Running VidGear Version: 0.2.6
[11/22/2022-00:02:20] [TRT] [I] Loaded engine size: 56 MiB
[11/22/2022-00:02:21] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in engine deserialization: CPU +0, GPU +58, now: CPU 0, GPU 58 (MiB)
[11/22/2022-00:02:21] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +1, GPU +52, now: CPU 1, GPU 110 (MiB)
[11/22/2022-00:02:21] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars
/content/simple-HigherHRNet/misc/HeatmapParser.py:168: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mod

Now check out the video output.avi
